In [9]:
from elasticsearch import Elasticsearch

In [10]:
es = Elasticsearch()

#name of the created index
index_name = "applications"

In [49]:
# Create index with settings and mapping

# This test is done during development only. 
if es.indices.exists(index_name):
    es.indices.delete(index=index_name)
    

# index settings
settings = {
  "settings": {
    "analysis": {
      "filter": {
        "trigrams_filter": {
          "type": "ngram",
          "min_gram": 3,
          "max_gram": 4
        }
      },
      "analyzer": {
        "text_processing": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "trigrams_filter"
          ]
        }
      }
    }
  }
,"mappings": {
    "properties": {
        "id": {
          "type": "keyword"
        },
        "title": {
          "type": "text"
        },
        "description": {
          "type": "text"
        },
        "StoreRating": {
            "type":"double"
        }
    }

}

    }
# create index
es.indices.create(index=index_name, ignore=400, body=settings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'applications'}

In [50]:
from elasticsearch import helpers
import gzip
import json

i = 0
# a list that will hold the bulks
actions = []
with gzip.open('scrapy01.json.gz','rt') as f:
    for line in f:
        i=i+1
        #if any line is not an application. such as the first line "["
        if len(line)<10:
            continue
        z = line.replace("\'", "")
        #removing the comma from the end of documents
        a = z[0:-2]
        #converting into dictionaries
        x = eval(a)
        if len(x['Title'])==0 or len(x['ReviewsAverage'])==0:
            continue
        #creating objects with the needed fields
        lo = {
            "title":x["Title"][0],
            "description":x["Description"],
            "StoreRating":x["ReviewsAverage"][0],
            "_id":x["Url"].split("&")[0]
        }
        #inserting data for bulks
        if i%1000!=0:
            actions.append(lo)
        else:
            #bulk insert
            try:
                helpers.bulk(es, actions,index=index_name)
            except:
                None
            actions=[]
        if i==1000000:
            break